In [0]:
%sql
USE pinterest_data;



In [0]:
/* SELECT description, COUNT(*) AS repeat_count
FROM df_pin
GROUP BY description
ORDER BY repeat_count DESC
LIMIT 10; */


In [0]:
CREATE OR REPLACE TEMP VIEW df_pin_clean AS
    SELECT 
        category,
        CASE 
            WHEN description IN ('No description available Story format', 'No description available') 
            THEN NULL
            ELSE description
        END AS description,
        downloaded,
        follower_count,
        image_src,
        `index`,
        is_image_or_video,
        poster_name,
        save_location,
        tag_list,
        title,
        unique_id
    FROM df_pin;

In [0]:
CREATE OR REPLACE TEMP VIEW df_pin_clean2 AS
  SELECT *
  FROM df_pin_clean
  WHERE follower_count != 'User Info Error';


In [0]:
/* SELECT image_src, COUNT(*) AS repeat_count
FROM df_pin_clean2
GROUP BY image_src
ORDER BY repeat_count DESC
LIMIT 10; */

In [0]:
CREATE OR REPLACE TEMP VIEW df_pin_clean3 AS
    SELECT 
        category,
        description,
        downloaded,
        follower_count,
        CASE 
            WHEN image_src IN ('Image src error.') 
            THEN NULL
            ELSE image_src
        END AS image_src,
        `index`,
        is_image_or_video,
        poster_name,
        save_location,
        tag_list,
        title,
        unique_id
    FROM df_pin_clean2;

In [0]:
/* SELECT tag_list, COUNT(*) AS repeat_count
FROM df_pin_clean3
GROUP BY tag_list
ORDER BY repeat_count DESC
LIMIT 10; */

In [0]:
CREATE OR REPLACE TEMP VIEW df_pin_clean4 AS
  SELECT 
      category,
      description,
      downloaded,
      follower_count,
      image_src,
      `index`,
      is_image_or_video,
      poster_name,
      save_location,
      CASE 
        WHEN tag_list IN ('N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e') 
        THEN NULL
        ELSE tag_list
      END AS tag_list,
      title,
      unique_id
  FROM df_pin_clean3;

In [0]:

CREATE OR REPLACE TEMP VIEW df_pin_clean5 AS
SELECT
    category,
    description,
    downloaded,
    CASE
      WHEN follower_count LIKE '%k' THEN CAST(CAST(SUBSTRING(follower_count, 1, LENGTH(follower_count) - 1) AS DECIMAL) * 1000 AS INT) -- removes 'k' from value, multiplies by 1000 and converts to INT
      WHEN follower_count LIKE '%M' THEN CAST(CAST(SUBSTRING(follower_count, 1, LENGTH(follower_count) - 1) AS DECIMAL) * 1000000 AS INT) -- removes 'm' from value, multiplies by 1000000 and converts to INT
      ELSE CAST(follower_count AS INT)
    END AS follower_count,
    image_src,
    `index`,
    is_image_or_video,
    poster_name,
    save_location,
    tag_list,
    title,
    unique_id
FROM df_pin_clean4;


In [0]:
CREATE OR REPLACE TEMP VIEW df_pin_clean6 AS
SELECT
    category,
    description,
    downloaded,
    follower_count,
    image_src,
    `index`,
    is_image_or_video,
    poster_name,
    TRIM(SUBSTRING(save_location, 16)) AS save_location,  -- removes 'Local save in' prefix
    tag_list,
    title,
    unique_id
FROM df_pin_clean5;

In [0]:
CREATE OR REPLACE TEMP VIEW df_pin_clean7 AS
SELECT
    category,
    description,
    downloaded,
    follower_count,
    image_src,
    `index` AS ind,  -- Renaming column 'index' to 'ind'
    is_image_or_video,
    poster_name,
    save_location,
    tag_list,
    title,
    unique_id
FROM df_pin_clean6;

In [0]:
CREATE OR REPLACE TABLE df_pin_cleaned AS
SELECT
    ind,
    unique_id,
    title,
    description,
    follower_count,
    poster_name,
    tag_list,
    is_image_or_video,
    image_src,
    save_location,
    category
FROM df_pin_clean7;

num_affected_rows,num_inserted_rows


In [0]:
DROP TABLE IF EXISTS df_pin;


In [0]:
CREATE OR REPLACE TABLE df_pin AS
SELECT *
FROM df_pin_cleaned;

num_affected_rows,num_inserted_rows


In [0]:
DROP TABLE IF EXISTS df_pin_cleaned;